In [86]:
import numpy as np
from numpy import pi, sqrt
import control
from control import matlab

![picture 1](0a1cea79ee38a9b462b6e5691e73c2f0d5b6328cee9285bbc72ee20383e786e1.png)  


![picture 2](6eabcd7fb450aaa0223e4f7b27111b3cb0bcfee523d36a00c6c79460cd4d72ed.png)  


Bac 1
$$\frac{dV1}{dt}=q_e-q_{12}$$
Bac 2
$$\frac{dV2}{dt}=q_{12}-q_{f2}-q_{23}$$
Bac 3
$$\frac{dV3}{dt}=q_{23}-q_{f3}$$

![picture 3](8f955475a4fa57b548c8acb10e43316fbc766fe3ac13c95f55994807918c49fc.png)  


On commence par poser les équations
$$q_e-q_{12}=0\longrightarrow x_1-x_2=\left(\frac{q_e}{q_{12}}\right)^2\qquad q_e=q_{12}$$
$$q_{12}-q_{f2}-q_{23}=0$$
$$q_{23}-q_{f3}=0\longrightarrow x_2=\left(\frac{a_{f}^2}{a_{23}^2}+1\right)x_3$$

On remplace les équations 1 et 3 dans l'équation 2
$$q_{e}-a_f\sqrt{x_2}-a_{23}\sqrt{x_2-x_3}=0$$
$$a_f\sqrt{x_2}+a_{23}\sqrt{x_2-x_3}=q_e$$
On remplace par l'expression de $x_2$ trouvée avec l'équation 3
$$a_f\sqrt{\left(\frac{a_{f}^2}{a_{23}^2}+1\right)x_3}+a_{23}\sqrt{\left(\frac{a_{f}^2}{a_{23}^2}+1\right)x_3-x_3}=q_e$$
$$a_f\sqrt{\left(\frac{a_{f}^2}{a_{23}^2}+1\right)}\sqrt{x_3}+a_{23}\sqrt{\left(\frac{a_{f}^2}{a_{23}^2}\right)}\sqrt{x_3}=q_e$$
$$\boxed{x_3=\frac{q_e^2}{\left(a_f\sqrt{\frac{a_f^2}{a_{23}^2}+1}+a_{23}\sqrt{\frac{a_f^2}{a_{23}^2}}\right)}}$$

Puis on calcule simplement les valeurs
$$x_3=0.1886$$
$$x_2=0.2219$$
$$x_1=0.2676$$


In [42]:
aD = 0.72
g = 9.81
qe = 100/1000 / 3600
S = 0.013

S12 = pi*(7.2e-3/2)**2
S23 = pi*(5.4e-3/2)**2 #ok
Sf = pi*(3.5e-3/2)**2 #ok

af = aD * Sf * sqrt(2*g)
a23 = aD * S23 * sqrt(2*g)
a12 = aD * S12 * sqrt(2*g)

x3eq = qe**2/(af*sqrt(af**2/a23**2+1) + a23*sqrt((af**2)/a23**2))**2
x2eq = (af**2/a23**2+1)*x3eq
x1eq = (qe / a12)**2 + x2eq
print(f"x1 : {x1eq:.4f} x2 : {x2eq:.4f} x3 : {x3eq:.4f}")

x1 : 0.2676 x2 : 0.2219 x3 : 0.1886


![picture 4](4f6561f85d6e42be69aac193b14310d9ef93409a262168a2078379bb718f8dd8.png)  


### Sur $x_1$
$$f(x_1,x_2)=-a_{12}\sqrt{x_1(t)-x_2(t)}$$

$$\frac{d\Delta x_1(t)}{dt}=\frac{1}{S}\left(\Delta u(t)+\frac{\partial f}{\partial x_1}(x_{1eq},x_{2eq})(x_1-x_{1eq}) +\frac{\partial f}{\partial x_2}(x_{1eq},x_{2eq})(x_2-x_{2eq})\right)$$

### Sur $x_2$

Voir le formulaire pour la linéarisation, il semblerait que le corrigé du prof soit faux...

### Sur $x_3$
$$f(x_2,x_3)=a_{23}\sqrt{x_2-x_3}-a_f\sqrt{x_3}$$
$$\frac{d\Delta x_3(t)}{dt}=\frac{1}{S}\left(\frac{\partial}{\partial x_2}f(x_{2eq},x_{3eq})(x_2-x_{2eq}+\frac{\partial}{\partial x_2}f(x_{2eq},x_{3eq})(x_2-x_{2eq}\right)

![picture 5](92dd001b854a704317eea5053d84211da79acc9e5f02385ae3344556bfaccc30.png)  


In [84]:
A = np.matrix('[-0.2415 0.2415 0;0.2415 -0.4244 0.1566;0 0.1566 -0.1865]')
B = np.matrix('[795.7747;0;0]')
C = np.matrix('[1 0 0]')

print(f"Valeurs propres (pôles en boucle ouverte) : {np.linalg.eig(A)[0]}")

Valeurs propres (pôles en boucle ouverte) : [-0.629879   -0.01797028 -0.20455072]


![picture 6](421399a2e10a43742b0a75109f147bce98e2e15e3eda863ab8aa610aa3fd6fd4.png)  


$$A-EC$$
$$(A-EC)^{T}$$
$$A^T-(EC)^T$$
$$A^T-C^TE^T$$

In [87]:
# On utilise la méthode de Ackermann A-BK -> A-EC
poles = 2*np.array([-0.6299]*3)
E = control.acker(A.T, C.T, poles).T
print(E)

# Vérification
print(f"Valeurs propres de A-EC : 2*{np.real(np.linalg.eig(A-E@C)[0]/2)}")

[[ 2.927     ]
 [11.71572124]
 [34.62664855]]
Valeurs propres de A-EC : [-0.62990369 -0.62989816 -0.62989816]


![picture 7](884bc13de39ba23f840c431265b8ce03e5ceb0ef6628869f4408fa74f5713fed.png)  


In [95]:
# On veut déterminer K dans A-BK
poles = 1/3*np.array([-0.6299]*3)
K = control.acker(A, B, poles)

print(f"K : {K}")

print(f"Valeurs propres de A-BK : 1/3*{np.real(np.linalg.eig(A-B@K)[0]*3)}")

K : [[-0.0002796   0.00064703 -0.00013606]]
Valeurs propres de A-BK : 1/3*[-0.62990534 -0.62989733 -0.62989733]
